In [1]:
import yfinance
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

In [2]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show nbformat

Name: nbformat
Version: 5.10.4
Summary: The Jupyter Notebook format
Home-page: https://jupyter.org
Author: 
Author-email: Jupyter Development Team <jupyter@googlegroups.com>
License: BSD 3-Clause License

- Copyright (c) 2001-2015, IPython Development Team
- Copyright (c) 2015-, Jupyter Development Team

All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior writte

In [7]:
ticker = "GOOG"
period = "1y"
interval = "1d"
data_df = yfinance.download(ticker, period=period, interval=interval)
data_df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GOOG,GOOG,GOOG,GOOG,GOOG
Date,,,,,
2024-03-18,147.780045,152.209064,147.441651,148.665848,47676700
2024-03-19,147.222687,148.914670,146.316973,148.277687,17748400
2024-03-20,148.974380,149.054005,146.968880,148.088577,17730000
2024-03-21,148.038818,150.591714,147.312249,149.611372,19843900
2024-03-22,151.054550,151.840819,149.382462,149.531764,19226300
...,...,...,...,...,...
2025-03-12,169.000000,169.529999,165.479996,168.470001,19880100
2025-03-13,164.729996,168.119995,164.070007,167.979996,15206200


In [9]:
# マルチインデックスカラムの処理
if isinstance(data_df.columns, pd.MultiIndex):
    # 第1レベル（Price, Close, High, Low, Open）だけをカラムとして使用
    data_df.columns = data_df.columns.get_level_values(0)

In [11]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=data_df.index,
            open=data_df["Open"],
            high=data_df["High"],
            low=data_df["Low"],
            close=data_df["Close"],
        ),
    ]
)

fig.update_layout(
    title="Google Stock Price",
    xaxis_title="Date",
    yaxis_title="Price(USD)",
)

fig.show()


In [18]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.3, subplot_titles=("ロウソク足チャート", "出来高"))
fig.add_trace(
    go.Candlestick( # ロウソク足チャート
        x=data_df.index,
        open=data_df["Open"],
        high=data_df["High"],
        low=data_df["Low"],
        close=data_df["Close"],
        name="Candlestick"
    ), row=1, col=1)
fig.add_trace( # 出来高の棒グラフ
    go.Bar(
        x=data_df.index,
        y=data_df["Volume"],
        name="Volume"
    ), row=2, col=1)
fig.update_layout(
    title="Google Stock Price",
    xaxis2_title="Date",
    yaxis_title="Price(USD)",
    xaxis_rangeslider_visible=False,
    xaxis2_rangeslider_visible=True
)
fig.show()

MACDとは  
- 短期EMA(指数平滑移動平均線)と長期EMAの差。一般的には、12日EMAと26日EMAの差を用いる。
- シグナルライン：MACDラインのEMA。一般的には、9日間のMACDラインのEMAを使用する。
- ヒストグラム：MACDラインとシグナルラインの差。MACDヒストグラムは市場の勢いを示し、MACDラインがシグナルラインを上回る時に正の値（強気）、下回る時に負の値（弱気）をとる。
- MACDラインがシグナルラインを超えてくるとき（MACDヒストグラムが正の値になる時）に買いシグナル。逆が売りシグナル。

In [19]:
short_ema = data_df["Close"].ewm(span=12, adjust=False).mean()
long_ema = data_df["Close"].ewm(span=26, adjust=False).mean()

macd = short_ema - long_ema
signal = macd.ewm(span=9, adjust=False).mean()
hist = macd - signal

data_df["MACD"] = macd
data_df["Signal"] = signal
data_df["Hist"] = hist

data_df

Price,Close,High,Low,Open,Volume,MACD,Signal,Hist
Date,,,,,,,,
2024-03-18,147.780045,152.209064,147.441651,148.665848,47676700,0.000000,0.000000,0.000000
2024-03-19,147.222687,148.914670,146.316973,148.277687,17748400,-0.044462,-0.008892,-0.035569
2024-03-20,148.974380,149.054005,146.968880,148.088577,17730000,0.060947,0.005075,0.055871
2024-03-21,148.038818,150.591714,147.312249,149.611372,19843900,0.068205,0.017701,0.050504
2024-03-22,151.054550,151.840819,149.382462,149.531764,19226300,0.313686,0.076898,0.236788
...,...,...,...,...,...,...,...,...
2025-03-12,169.000000,169.529999,165.479996,168.470001,19880100,-5.785001,-5.312706,-0.472295
2025-03-13,164.729996,168.119995,164.070007,167.979996,15206200,-6.013631,-5.452891,-0.560740
2025-03-14,167.619995,168.250000,164.509995,165.315002,18611100,-5.893684,-5.541050,-0.352634


In [20]:
fig  = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.3, subplot_titles=("ロウソク足チャート", "MACD"))
fig.add_trace(
    go.Candlestick( # ロウソク足チャート
        x=data_df.index,
        open=data_df["Open"],
        high=data_df["High"],
        low=data_df["Low"],
        close=data_df["Close"],
        name="Candlestick"
    ), row=1, col=1)
fig.add_trace( # MACD
    go.Scatter(
        x=data_df.index,
        y=data_df["MACD"],
        mode="lines",
        name="MACD"
        ), row=2, col=1)
fig.add_trace( # Signal
    go.Scatter(
        x=data_df.index,
        y=data_df["Signal"],
        mode="lines",
        name="Signal"
        ), row=2, col=1)
fig.add_trace( # Hist
    go.Bar(
        x=data_df.index,
        y=data_df["Hist"],
        name="Hist"
    ), row=2, col=1)
fig.update_layout(
    title="Google Stock Price",
    xaxis2_title="Date",
    yaxis_title="Price(USD)",
    yaxis2_title="MACD",
    xaxis_rangeslider_visible=False,
    xaxis2_rangeslider_visible=True
)
fig.show()
